In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import holidays
from typing import Optional

In [2]:
file_path = './data/'
TICKER = '^GSPC'
TIME_FRAME = '1d'

In [3]:
# To use Christian's daily futures data, set USE_FUTURES_DAILY_DATA to True.  To use the Yahoo Finance daily data, set USE_FUTURES_DAILY_DATA to False.
USE_FUTURES_DAILY_DATA = True

# Turn gap on by setting USE_GAP to True.  Turn it off by setting USE_GAP to False.
USE_GAP = True

In [4]:
# Get the data
if USE_FUTURES_DAILY_DATA:
    df = pd.read_csv('./data/futures_d.csv')

else:
    df = pd.read_csv(f'./data/{TICKER}_{TIME_FRAME}.csv')
df.Date = pd.to_datetime(df.Date)
df['Date'] = df['Date'].dt.normalize()
df = df[(df['Date'] >= '2019-08-05')]  # ALL DATA
# df = df[(df['Date'] >= '2020-02-01') & (df['Date'] <= '2020-06-15')]  # COVID
# df = df[(df['Date'] >= '2020-08-01')]  # BULL
df.reset_index(drop=True, inplace=True)
us_holidays = holidays.US()
df['Holiday'] = df['Date'].apply(lambda x: 1 if x.strftime('%Y-%m-%d') in us_holidays else 0)

df

,Symbol,Date,EMA,100 EMA,50 EMA,8 EMA,200 SMA,VOL,5 Day AT,5 Day AT.1,10 Day A,10 Day A.1,Open,High,Low,Close,Volume,Holiday
0,ESU24,2019-08-05,3249.25,3185.50,3228.25,3225.75,3131.75,2952153,64.00,114.00,43.25,114.00,3211.00,3215.25,3102.25,3106.50,2952153,0
1,ESU24,2019-08-06,3240.50,3185.00,3225.25,3210.50,3132.00,2531815,80.50,108.00,52.00,108.00,3098.00,3165.50,3057.50,3157.25,2531815,0
2,ESU24,2019-08-07,3233.00,3184.50,3222.75,3199.75,3132.25,2585719,81.50,68.25,56.25,68.25,3155.50,3173.25,3105.00,3161.75,2585719,0
3,ESU24,2019-08-08,3232.00,3185.25,3222.75,3204.25,3133.00,1730455,81.50,70.25,60.25,70.25,3160.75,3222.50,3152.25,3220.75,1730455,0
4,ESU24,2019-08-09,3229.50,3185.75,3222.00,3204.50,3133.75,1962653,80.00,40.00,61.75,40.00,3209.75,3217.50,3180.75,3205.50,1962653,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1272,ESU24,2024-08-06,5444.00,5299.75,5399.75,5398.25,5119.50,2072145,131.25,120.25,88.25,120.25,5250.00,5342.00,5221.75,5249.00,2072145,0
1273,ESU24,2024-08-07,5421.00,5297.75,5392.00,5354.75,5120.25,2137837,156.50,162.50,99.75,162.50,5241.50,5359.25,5196.75,5202.25,2137837,0
1274,ESU24,2024-08-08,5414.50,5299.00,5390.50,5354.25,5122.75,1829194,183.00,179.00,114.75,179.00,5208.25,5361.00,5182.00,5353.00,1829194,0
1275,ESU24,2024-08-09,5409.75,5300.25,5389.50,5356.75,5125.00,1348468,153.00,65.75,117.75,65.75,5348.25,5385.25,5319.50,5365.00,1348468,0


In [5]:
df_small = pd.read_csv(f'./data/ESU24.CME_5m.csv')
df_small.Date = pd.to_datetime(df_small.Date)
df_small = df_small[df_small['Date'] >= df['Date'][0]]
df_small.reset_index(drop=True, inplace=True)

df_small

,ESU24,Date,Open,High,Low,Last,Volume
0,ESU24,2019-08-05 00:00:00,3182.25,3183.75,3182.00,3183.25,1462
1,ESU24,2019-08-05 00:05:00,3183.25,3185.25,3181.75,3184.75,5295
2,ESU24,2019-08-05 00:10:00,3184.75,3186.00,3184.25,3185.75,3781
3,ESU24,2019-08-05 00:15:00,3185.75,3186.50,3184.75,3186.50,3833
4,ESU24,2019-08-05 00:20:00,3186.50,3187.50,3184.75,3185.75,3510
...,...,...,...,...,...,...,...
348156,ESU24,2024-08-08 12:45:00,5331.25,5348.50,5331.00,5347.25,21807
348157,ESU24,2024-08-08 12:50:00,5347.25,5347.75,5338.25,5342.00,20451
348158,ESU24,2024-08-08 12:55:00,5342.00,5347.50,5337.25,5341.25,32271
348159,ESU24,2024-08-08 13:00:00,5341.25,5352.50,5334.00,5347.25,93713


In [6]:
class Trade:
    def __init__(self, trade_type: str, open_price: float, sl: float, sg: float, n_lots: int) -> None:
        self.trade_type = trade_type
        self.open_price = open_price
        self.sl = sl
        self.sg = sg
        self.n_lots = n_lots

    def is_buy(self) -> bool:
        return self.trade_type == 'buy'

In [7]:
class SimulationResults:
    def __init__(self) -> None:
        self.points = 0
        self.n_wins = 0
        self.n_losses = 0
        self.win_streak = 0
        self.loss_streak = 0
        self.curr_win_streak = 0
        self.curr_loss_streak = 0
        self.n_buys = 0
        self.n_sells = 0
        self.n_ticks = 0
        self.dollars = 0

    def update(self, trade: Trade, trade_amount: float) -> None:
        self.n_buys += 1 if trade.is_buy() else 0
        self.n_sells += 1 if not trade.is_buy() else 0
        self.points += trade_amount
        n_ticks = trade_amount * 4
        dollars = n_ticks * 1.25 * trade.n_lots
        self.n_ticks += n_ticks
        self.dollars += dollars - 0.4
        self.n_wins += 1 if trade_amount > 0 else 0
        self.n_losses += 1 if trade_amount < 0 else 0
        self.curr_win_streak = (self.curr_win_streak + 1) if trade_amount > 0 else 0
        self.curr_loss_streak = (self.curr_loss_streak + 1) if trade_amount < 0 else 0
        self.win_streak = max(self.win_streak, self.curr_win_streak)
        self.loss_streak = max(self.loss_streak, self.curr_loss_streak)

    def __str__(self) -> str:
        return f'Points: {self.points}\nTicks won: {self.n_ticks}\nDollars won: {self.dollars}\nNum buys: {self.n_buys}\nNum sells: {self.n_sells}\nNum wins: {self.n_wins}\nNum losses: {self.n_losses}\nWin streak: {self.win_streak}\nLoss streak: {self.loss_streak}'

In [8]:
def run_simulation(points_diff_p: int, stop_gain_points_p: int, stop_loss_points_p: int, remove_holidays: bool, n_lots: int, can_buy: bool, can_sell: bool, gap: bool, reverse: bool) -> SimulationResults:
    place_trade, buy, sim_results = False, False, SimulationResults()
    points_diff_p = points_diff_p / 100
    sg_points_p, sl_points_p = stop_gain_points_p / 100, stop_loss_points_p / 100
    i = 1 if gap else 0

    def _iterate_through_trade(start_date: datetime, place_buy: bool) -> Optional[datetime]:
        # new_date = start_date.replace(hour=13, minute=30)
        new_date = start_date.replace(hour=6, minute=30) if gap else start_date.replace(hour=15, minute=0) 
        curr_df_small = df_small[df_small['Date'] >= new_date]
        curr_df_small.reset_index(drop=True, inplace=True)

        if len(curr_df_small) == 0:
            return None

        open_price = curr_df_small.loc[curr_df_small.index[0], 'Open']

        if place_buy:
            sl = open_price + 1 - (sl_points_p * open_price)
            sg = open_price + 1 + (sg_points_p * open_price)
            trade = Trade('buy', open_price, sl, sg, n_lots)

        else:
            sl = open_price - 1 + (sl_points_p * open_price)
            sg = open_price - 1 - (sg_points_p * open_price)
            trade = Trade('sell', open_price, sl, sg, n_lots)


        for j in range(len(curr_df_small)):
            curr_date, curr_low, curr_high = curr_df_small.loc[curr_df_small.index[j], ['Date', 'Low', 'High']]

            if trade.is_buy():
                if curr_low <= trade.sl:
                    trade_amount = trade.sl - trade.open_price
                    sim_results.update(trade, trade_amount)
                    return curr_date

                if curr_high >= trade.sg:
                    trade_amount = trade.sg - trade.open_price
                    sim_results.update(trade, trade_amount)
                    return curr_date

            else:
                if curr_high >= trade.sl:
                    trade_amount = trade.open_price - trade.sl
                    sim_results.update(trade, trade_amount)
                    return curr_date

                if curr_low <= trade.sg:
                    trade_amount = trade.open_price - trade.sg
                    sim_results.update(trade, trade_amount)
                    return curr_date
                
        return None

    while i < len(df):
        if not place_trade:
            date, curr_open, curr_close, is_holiday = df.loc[df.index[i], ['Date', 'Open', 'Close', 'Holiday']]
            if gap:
                curr_close = df.loc[df.index[i - 1], 'Close']
            not_a_holiday = not (remove_holidays and is_holiday)
            buy_condition = curr_close > curr_open if reverse else curr_close < curr_open
            sell_condition = curr_close < curr_open if reverse else curr_close > curr_open

            if can_buy and buy_condition and abs(curr_open - curr_close) >= points_diff_p * curr_close and not_a_holiday:
                place_trade, buy = True, True

            elif can_sell and sell_condition and abs(curr_open - curr_close) >= points_diff_p * curr_close and not_a_holiday:
                place_trade, buy = True, False

        if place_trade:
            trade_end_date = _iterate_through_trade(date, buy)
            place_trade, buy = False, False

            if trade_end_date is None:
                break

            else:
                i = int(df.loc[df['Date'] <= trade_end_date].index[-1] + 1)

        else:
            i += 1

    return sim_results

In [9]:
num_points_p = [0.1, 0.5, 1, 2]
sg_points_p = [0.1, 0.5, 1, 2]
sl_points_p = [0.1, 0.5, 1, 2]
remove_holidays_vals = [False]
can_buy_vals = [True, False]
can_sell_vals = [True, False]
reverse_vals = [True, False]
all_combos = []

for n_points in num_points_p:
    for sg_point in sg_points_p:
        for sl_point in sl_points_p:
            for remove_holiday in remove_holidays_vals:
                for can_b in can_buy_vals:
                    for can_s in can_sell_vals:
                        for reverse in reverse_vals:
                            if not can_b and not can_s:
                                continue
                            all_combos.append((n_points, sg_point, sl_point, remove_holiday, can_b, can_s, reverse))

best_num_points = None
best_sg_points = None
best_sl_points = None
best_remove_holidays_vals = None
best_can_buy_vals = None
best_can_sell_vals = None
best_reverse_val = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished, n_runs = 0, len(all_combos)
best_rewards = []
best_results = None

print(f'N runs: {n_runs}')

for n_points, sg_point, sl_point, remove_holiday, can_b, can_s, reverse in all_combos:
    results = run_simulation(points_diff_p=n_points, stop_gain_points_p=sg_point, stop_loss_points_p=sl_point, remove_holidays=remove_holiday, n_lots=1, can_buy=can_b, can_sell=can_s, gap=USE_GAP, reverse=reverse)
    profit = results.dollars
    runs_finished += 1

    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None
    if min_item is None or profit > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': profit, 'n_points': n_points, 'sg_points': sg_point, 'sl_points': sl_point, 'remove_holiday': remove_holiday, 'can_buy': can_b, 'can_sell': can_s, 'reverse': reverse})

    if profit > best_reward:
        best_num_points = n_points
        best_sg_points = sg_point
        best_sl_points = sl_point
        best_remove_holidays_vals = remove_holiday
        best_can_buy_vals = can_b
        best_can_sell_vals = can_s
        best_reverse_val = reverse
        best_results = results
        best_reward = profit


    print('Best profit so far: ' + str(best_reward) + '\n')

N runs: 384
Remaining runs: 383
Best profit so far: -711.7674999999897

Remaining runs: 382
Best profit so far: -220.8949999999966

Remaining runs: 381
Best profit so far: -220.8949999999966

Remaining runs: 380
Best profit so far: -116.20249999999106

Remaining runs: 379
Best profit so far: -116.20249999999106

Remaining runs: 378
Best profit so far: -104.6925000000058

Remaining runs: 377
Best profit so far: -104.6925000000058

Remaining runs: 376
Best profit so far: -104.6925000000058

Remaining runs: 375
Best profit so far: -104.6925000000058

Remaining runs: 374
Best profit so far: 173.58749999998776

Remaining runs: 373
Best profit so far: 173.58749999998776

Remaining runs: 372
Best profit so far: 173.58749999998776

Remaining runs: 371
Best profit so far: 173.58749999998776



KeyboardInterrupt: 

In [ ]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best distance points: ' + str(best_num_points))
print('Best sg points: ' + str(best_sg_points))
print('Best sl points: ' + str(best_sl_points))
print('Best remove holiday val: ' + str(best_remove_holidays_vals))
print('Best can buy val: ' + str(best_can_buy_vals))
print('Best can sell val: ' + str(best_can_sell_vals))
print('Best reverse val: ' + str(best_reverse_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 9593.86875000001
Best distance points: 0.1
Best sg points: 0.5
Best sl points: 2
Best remove holiday val: False
Best can buy val: True
Best can sell val: False
-----------------------
Top results:
{'reward': 4056.535000000008, 'n_points': 0.1, 'sg_points': 0.1, 'sl_points': 2, 'remove_holiday': False, 'can_buy': True, 'can_sell': False}
{'reward': 4807.806249999961, 'n_points': 0.1, 'sg_points': 0.5, 'sl_points': 1, 'remove_holiday': False, 'can_buy': True, 'can_sell': True}
{'reward': 5758.381249999996, 'n_points': 0.1, 'sg_points': 0.5, 'sl_points': 1, 'remove_holiday': False, 'can_buy': True, 'can_sell': False}
{'reward': 5934.006249999957, 'n_points': 0.1, 'sg_points': 0.5, 'sl_points': 2, 'remove_holiday': False, 'can_buy': True, 'can_sell': True}
{'reward': 9593.86875000001, 'n_points': 0.1, 'sg_points': 0.5, 'sl_points': 2, 'remove_holiday': False, 'can_buy': True, 'can_sell': False}
{'reward': 7603.287500000106, 'n_points': 0

In [ ]:
print('Best results:')
print(best_results)

Best results:
Points: 1947.9737499999924
Ticks won: 7791.8949999999695
Dollars won: 9593.86875000001
Num buys: 365
Num sells: 0
Num wins: 309
Num losses: 56
Win streak: 21
Loss streak: 2
